In [ ]:
import torch
import os
import torchvision
from torch import nn
import torchvision.models as models
from torchvision.models import resnet34, vgg13, alexnet, squeezenet1_1, mobilenet_v2
from torchvision import datasets
import torchvision.transforms as transforms
import neptune.new as neptune
import torch.optim as optim
import simsiam.loader
import time
from torch.backends import cudnn
import numpy as np
import random

Hiper-parámetros que tiene que definir el usuario

In [ ]:
arch = 'mobilenet_v2' #Opciones: resnet34, mobilenet_v2, alexnet, vgg13 y squeezenet1_1
dataset = 'ImageWoof' #Nombre del dataset
train_path = 'ImageWoof/train' #Path del dataset con las imágenes de train
val_path = 'ImageWoof/val' #Path del dataset con las imágenes de val
model_path = 'Classifier_Checkpoints/ImageWoof/Mobilenet/' #Path donde se querrá guardar el modelo
weight_decay = 0.00004
momentum = 0.9
n_epochs = 300 #Número de epocas
init_lr = 0.045 #Learning rate inicial
batch_size = 64
lr_scheduler = 'linear' #Como decrecer el lr. Opciones: '', 'linear', 'plateau', '98'
nombre = 'SupervisedImageWoofMobilenetV2' #Nombre del experimento
gpu = 0 #Número de la gpu donde realizar el experimento

Código

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
def adjust_learning_rate(optimizer, init_lr, epoch, n_epochs, lr_scheduler):
    if lr_scheduler == 'linear':
        cur_lr = -(init_lr/n_epochs)*epoch + init_lr
    elif lr_scheduler == '98':
        cur_lr = init_lr*pow(0.98, epoch)
    else:
        cur_lr = init_lr
    for param_group in optimizer.param_groups:
            param_group['lr'] = cur_lr

In [ ]:
def get_backbone(backbone_name):
        return {'resnet34': resnet34(num_classes=10),
                'alexnet': alexnet(num_classes=10),
                'squeezenet1_1': squeezenet1_1(num_classes=10),
                'mobilenet_v2': mobilenet_v2(num_classes=10),
                'vgg13': vgg13(num_classes=10)}[backbone_name]

In [ ]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))
    
    model.train(True)
    end = time.time()
    for i, (inputs, labels) in enumerate(train_loader):
        data_time.update(time.time() - end)
        
        inputs = inputs.cuda(0, non_blocking=True)
        labels = labels.cuda(0, non_blocking=True)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(acc1[0], inputs.size(0))
        top5.update(acc5[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if i % 10 == 0:
            progress.display(i)
    
    return losses.avg, top1.avg, top5.avg

In [ ]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()
    with torch.no_grad():
        end = time.time()
        for i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.cuda(0, non_blocking=True)
            labels = labels.cuda(0, non_blocking=True)

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1[0], inputs.size(0))
            top5.update(acc5[0], inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    return losses.avg, top1.avg, top5.avg

In [ ]:
model = get_backbone(arch)
criterion = nn.CrossEntropyLoss().cuda(0)
optimizer = optim.SGD(model.parameters(), lr=init_lr, momentum=momentum, weight_decay=weight_decay)
if lr_scheduler == 'plateau':
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, min_lr=0.001)
if gpu is not None:
    torch.cuda.set_device(gpu)
    model = model.cuda(gpu)
    cudnn.benchmark = True
print(model)

In [ ]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224),
     transforms.RandomHorizontalFlip(),
     transforms.ColorJitter(0.4,0.4,0.4,0.1),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
transform_val = transforms.Compose(
    [transforms.Resize((256,256)),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
train_set = datasets.ImageFolder(train_path, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_set = datasets.ImageFolder(val_path, transform=transform_val)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
best_val_top1 = 0.
best_val_top5 = 0.
torch.cuda.empty_cache()
for epoch in range(n_epochs):
    adjust_learning_rate(optimizer, init_lr, epoch, n_epochs, lr_scheduler)
    train_loss, train_top1, train_top5 = train(train_loader, model, criterion, optimizer, epoch)
    val_loss, val_top1, val_top5 = validate(val_loader, model, criterion)  
    if val_top1 > best_val_top1:
        best_val_top1 = val_top1
        name = 'best.pth'
        torch.save(model, model_path + name)
    if val_top5 > best_val_top5:
        best_val_top5 = val_top5
    if epoch % 5 == 4:
        name = 'checkpoint_{}.pth'.format(epoch)
        torch.save(model, model_path + name)
    if lr_scheduler == 'plateau':
        scheduler.step(val_loss)